# 第12章：アンサンブル

## 12.1 アンサンブルのメリットとデメリットを挙げよう。

### 【解答例】

#### メリット

- 複数の学習器を使う為、未学習のデータに対する予測能力を向上できる

#### デメリット

- 計算量が多くなるため、結果が出るまで多少時間がかかる
- 解釈性に劣る場合がある（どのようなロジックで結果が出てきたのかを説明するのが難しい）

### 【解説】

- Kaggleなどのコンペではアンサンブル（XGBoost、LightGBM等のブースティング）が使われることが一般的になってきました。これは、デメリットとして挙げた計算量についてあまり気にしなくて良いほど計算機が高性能化したことにも起因しています。
- 一方で、解釈性に劣りブラックボックス化しやすいという課題は依然として残っています。そこをどのように克服すれば良いか、是非考えてみましょう。

## 12.2 第8章の章末問題で自作した特徴量を使ってLightGBMモデルを作って精度を検証してみよう。

### 【解答例】

In [ ]:
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from sklearn.model_selection import train_test_split # ホールドアウト用モジュール
from sklearn.metrics import accuracy_score, roc_auc_score # 評価指標用モジュール

In [ ]:
# 前処理済み中間データのdictを読み取る
shomatsu_pp_data_dict = joblib.load("../intermediate/shomatsu_pp_data_dict.pkl3")
# 辞書型変数の値に格納された決定木向け中間データを読み取る
shomatsu_dtc_train_df = shomatsu_pp_data_dict["dtc"]["train"]
shomatsu_dtc_test_df = shomatsu_pp_data_dict["dtc"]["test"]
# 6:4の割合でホールドアウト法を行う
shomatsu_dtc_train_train_df, shomatsu_dtc_train_valid_df = train_test_split(
    shomatsu_dtc_train_df,
    test_size=0.4,
    random_state=57,
    shuffle=True
)
shomatsu_dtc_train_df.shape, shomatsu_dtc_train_train_df.shape, shomatsu_dtc_train_valid_df.shape, shomatsu_dtc_test_df.shape

In [ ]:
# 目的変数をtargetという変数に格納する
target = "buy_flag"
# 説明変数をfeaturesという変数に格納する
shomatsu_lgbc_features = shomatsu_dtc_train_df.columns.tolist()
# customer_idとbuy_flagは説明変数ではない為削除する
shomatsu_lgbc_features.remove("customer_id")
shomatsu_lgbc_features.remove("buy_flag")

In [ ]:
import lightgbm as lgb # LightGBM

In [ ]:
# LightGBM用データセットを生成する
lgb_train_train_dataset = lgb.Dataset(shomatsu_dtc_train_train_df[shomatsu_lgbc_features], shomatsu_dtc_train_train_df[target])
lgb_train_valid_dataset = lgb.Dataset(shomatsu_dtc_train_valid_df[shomatsu_lgbc_features], shomatsu_dtc_train_valid_df[target], reference=lgb_train_train_dataset)

lgb_train_dataset = lgb.Dataset(shomatsu_dtc_train_df[shomatsu_lgbc_features], shomatsu_dtc_train_df[target])
lgb_test_dataset = lgb.Dataset(shomatsu_dtc_test_df[shomatsu_lgbc_features], reference=lgb_train_dataset)

In [ ]:
# 予め設定しておくハイパーパラメータ
lgbc_params = {
    "objective": "binary", # 問題設定: 二値分類
    "metric": "auc", # 評価関数: AUC
    "verbosity": -1 # 出力なし
}

In [ ]:
# 学習する
lgbc_clf = lgb.train(
    lgbc_params, # ハイパーパラメータ
    train_set=lgb_train_train_dataset, # 学習データ
    num_boost_round=100, # ブースティングを行う回数
    verbose_eval=20 # ブースティング20回につき1回結果を出力
)

In [ ]:
# LightGBM用データセットを生成する
shomatsu_lgb_train_train_dataset = lgb.Dataset(
    shomatsu_dtc_train_train_df[shomatsu_lgbc_features],
    shomatsu_dtc_train_train_df[target]
)
shomatsu_lgb_train_valid_dataset = lgb.Dataset(
    shomatsu_dtc_train_valid_df[shomatsu_lgbc_features],
    shomatsu_dtc_train_valid_df[target],
    reference=shomatsu_lgb_train_train_dataset
)

shomatsu_lgb_train_dataset = lgb.Dataset(shomatsu_dtc_train_df[shomatsu_lgbc_features], shomatsu_dtc_train_df[target])
shomatsu_lgb_test_dataset = lgb.Dataset(shomatsu_dtc_test_df[shomatsu_lgbc_features], reference=shomatsu_lgb_train_dataset)

In [ ]:
!pip install --quiet optuna

In [ ]:
import optuna.integration.lightgbm as optuna_lgb # OptunaによるLightGBM
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# OptunaによるLightGBMのハイパーパラメータチューニング
shomatsu_optuna_lgbc_clf = optuna_lgb.train(
    lgbc_params, # 固定のハイパーパラメータ
    train_set=shomatsu_lgb_train_train_dataset, # 学習データ
    valid_sets=shomatsu_lgb_train_valid_dataset, # 検証データ
    num_boost_round=100, # boostingを行う回数
    verbose_eval=20, # ブースティング20回につき1回結果を出力
    optuna_seed=57 # 再現性確保のためseed値を指定
)

In [ ]:
# 最適なパラメータの確認
shomatsu_best_lgbc_params = shomatsu_optuna_lgbc_clf.params
shomatsu_best_lgbc_params

In [ ]:
# 学習する
shomatsu_lgbc_clf = lgb.train(
    shomatsu_best_lgbc_params, # 最適なハイパーパラメータ
    train_set=shomatsu_lgb_train_train_dataset, # 学習データ
    # valid_sets=shomatsu_lgb_valid_dataset, # 検証データ
    num_boost_round=100, # boostingを行う回数
    verbose_eval=20 # ブースティング20回につき1回結果を出力
)

In [ ]:
# train-validの予測する
shomatsu_lgbc_train_valid_proba_y = shomatsu_lgbc_clf.predict(shomatsu_dtc_train_valid_df[shomatsu_lgbc_features])
shomatsu_lgbc_train_valid_proba_y

In [ ]:
# train-validの検証する（AUCを算出する）
shomatsu_lgbc_train_valid_auc_val = roc_auc_score(
    shomatsu_dtc_train_valid_df[target],
    shomatsu_lgbc_train_valid_proba_y
)
shomatsu_lgbc_train_valid_auc_val

In [ ]:
# trainの予測する
shomatsu_lgbc_train_proba_y = shomatsu_lgbc_clf.predict(shomatsu_dtc_train_df[shomatsu_lgbc_features])
shomatsu_lgbc_train_pred_y = np.where(shomatsu_lgbc_train_proba_y >= 0.5, 1, 0)

# trainの検証する
shomatsu_lgbc_train_accuracy_val = accuracy_score(shomatsu_dtc_train_df[target], shomatsu_lgbc_train_pred_y)
shomatsu_lgbc_train_auc_val = roc_auc_score(shomatsu_dtc_train_df[target], shomatsu_lgbc_train_proba_y)
shomatsu_lgbc_train_accuracy_val, shomatsu_lgbc_train_auc_val

In [ ]:
# testの予測する
shomatsu_lgbc_test_proba_y = shomatsu_lgbc_clf.predict(shomatsu_dtc_test_df[shomatsu_lgbc_features])

In [ ]:
# sample submitデータを読み込む
gi_sample_submit_df = pd.read_csv("../input/gi_sample_submit.csv")

In [ ]:
# submit向けDataFrameを作成し、列に予測確率を格納する
shomatsu_submit_df = shomatsu_dtc_test_df.copy()[["customer_id"]]
shomatsu_submit_df["buy_proba"] = shomatsu_lgbc_test_proba_y
shomatsu_submit_df.head()

In [ ]:
gi_sample_submit_df.shape, shomatsu_submit_df.shape

In [ ]:
shomatsu_submit_df = pd.merge(gi_sample_submit_df.drop("buy_proba", axis=1), shomatsu_submit_df, on="customer_id", how="left").reset_index(drop=True)
shomatsu_submit_df.head()

In [ ]:
shomatsu_submit_df.shape

In [ ]:
# outputディレクトリにsubmit⽤ファイルを出⼒する
shomatsu_submit_df.to_csv(f"../output/shomatsu_submit_lgbc.csv", encoding="utf-8", index=False)

### 【解説】

- LightGBMに関しても、特徴量さえ出来てしまえば、モデルを構築して精度を検証するプロセス（プログラム）はほぼ同じものを流用することが出来ます。